In [1]:
from src.make_dataset import make_dataset

c:\ProgramData\Miniconda3\envs\bio\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
To use the Graphein submodule graphein.protein.features.sequence.embeddings, you need to install: biovec 
biovec cannot be installed via conda
To use the Graphein submodule graphein.protein.visualisation, you need to install: pytorch3d 
To do so, use the following command: conda install -c pytorch3d pytorch3d


[08/13/22 16:22:58] WARNING  To use the Graphein submodule graphein.protein.meshes, you need to        ]8;id=278935;file://c:\ProgramData\Miniconda3\envs\bio\lib\site-packages\graphein\protein\meshes.py\meshes.py]8;;\:]8;id=16800;file://c:\ProgramData\Miniconda3\envs\bio\lib\site-packages\graphein\protein\meshes.py#29\29]8;;\
                             install: pytorch3d                                                                    
                             To do so, use the following command: conda install -c pytorch3d pytorch3d             

In [3]:
make_dataset(index_xlsx="./index.xlsx", root="dataset")

3931it [9:07:12, 17.86s/it]

Protein:  3se4 Mutation:  LA116A


Output()